# Load libraries and summary functions

In [372]:
#Reference http://blog.kaggle.com/2016/12/27/a-kagglers-guide-to-model-stacking-in-practice/

In [373]:
#In this example when we a fit base model during cv we also run the prediction on the test data
#We then take a vote to from all the base cv models to make a prediction for the test data
#(This is mentioned in the link above)

In [374]:
suppressMessages({
library(dplyr) # for data manipulation
library(caret) # for model-building
library(pROC) # for AUC calculations
library(PRROC) # for Precision-Recall curve calculations
library(magrittr) # for pipes
})

In [375]:
options(repr.matrix.max.cols=50)
options(repr.matrix.max.rows=50)

In [376]:
set.seed(45406)

In [377]:
#Class1 == 0
#Calss2 == 1

aucSummary <- function(data, lev = NULL, model = NULL){
  
  index_class2 <- data$obs == "Class2"
  index_class1 <- data$obs == "Class1"
  
  #calc the auc metrics
  pr <- pr.curve(data$Class2[index_class2],
                 data$Class2[index_class1],
                 curve = FALSE)
  
  roc <- roc.curve(data$Class2[index_class2],
                   data$Class2[index_class1],
                   curve = FALSE)
  
  pr_out <- pr$auc.integral
  
  roc_out <- roc$auc
  
  obs <- data$obs
  
  
  #Assign threshold
  pre <- ifelse(data$Class2 > .5, 'Class2', 'Class1')
  
  #Calculate Accuracy  
  acc <- mean(obs == pre)
  
  out <- c(pr_out,roc_out,acc)  
  
  names(out) <-c("AUPRC","AUROC","Accuracy")
  
  out  
}

In [378]:
#Returns non empty sting with most occurrences in a vector 
ModeChar <- function(x){
  
  #Get rid of NA or empty values
  x <- x[!(x %in% c('',' ','NA','N/A','na','n/a'))]

  
  dd <- unique(x)
  D <- dd[which.max(tabulate(match(x,dd)))]
  
    
  if(is.na(D)){
    D <- ' '
    as.character(D)
  }
  if(!is.na(D)){
    as.character(D)
  }
   
}

In [379]:
#Returns non empty vector with the most occurrences from rows in a df
ModeRow <- function(df){
  
  length <- dim(df)[1]
    
  ModeCol <- c()  

  for(x in 1:length){
    
     Mode <-  ModeChar( df[x,] )
      
     ModeCol <-c(ModeCol,Mode) 
  }
  
  ModeCol
}

# Create test/train data sets

In [380]:
#Create data set
data_set <- twoClassSim(1000,
                    intercept = -6,
                    linearVars = 8,
                    noiseVars = 4)

In [381]:
#Create train/tests sets
index <- createDataPartition(data_set$Class, p = 6/10, list=FALSE)

train_set <- data_set[index,]
test_set <- data_set[-index,]

In [382]:
#Look at dim
train_set %>% dim()

train_set %>% head()

[1] 601  18

,TwoFactor1,TwoFactor2,Linear1,Linear2,Linear3,Linear4,Linear5,Linear6,Linear7,Linear8,Nonlinear1,Nonlinear2,Nonlinear3,Noise1,Noise2,Noise3,Noise4,Class
2,0.8097223,1.47437548,-0.8076689,-1.1319747,-0.2866550,1.78247610,0.16440952,0.26069540,0.5582444,-0.8972071,-0.04248304,0.9344437,0.91795414,0.8458606,1.7498821,1.6652243,0.5959273,Class1
3,1.2387178,0.01599239,-1.2053774,-1.0802625,2.2725103,-0.31690291,1.07191422,-0.52142051,1.0209103,0.8011080,-0.68616298,0.3735412,0.60716965,-0.6950674,-0.1531351,-0.1616989,1.7063336,Class1
5,-0.5131204,0.25986890,1.5314869,-0.4635079,1.5407780,-1.36485763,-2.01751135,-0.04614105,0.2751114,0.4361813,0.44862104,0.8372881,0.91693135,-0.1913738,0.6197387,-1.2638430,-0.1050231,Class2
6,1.0278531,1.61567486,-0.1555739,0.4161480,-0.1254957,0.07546147,0.93962369,-1.52580671,-0.1069174,-0.7167698,-0.66104746,0.4615281,0.14162099,-1.3987289,0.8233420,0.5832635,1.0977854,Class2
7,1.3514164,0.15930289,-0.9060178,0.9092948,-0.0106088,-0.17534387,0.06520887,0.16230173,-0.9382551,-1.0432937,0.01796992,0.9273337,0.18118455,-0.2059794,0.3343527,-0.5753191,-0.9176470,Class1
8,-0.3416407,-3.37577238,0.5900949,2.0505353,0.2392311,0.53155128,-0.68023745,1.96639816,1.4230917,1.2681464,-0.95171187,0.8373439,0.05076276,1.6775790,-1.4540916,0.1381279,-0.2966907,Class1


# Use cv to find the best parameters for thebase models


In [383]:
#Independent and dependent variables
X = 1:17
Y = 'Class'

In [384]:
ctrl <- trainControl(method = "repeatedcv",
                     number = 5,
                     repeats = 2,
                     summaryFunction = aucSummary,
                     classProbs = TRUE)

In [385]:
base_rf <- train(x=train_set[,X],
      y=train_set[,Y],
      method = "rf",
      trControl = ctrl,
      metric = "Accuracy",
      allowParallel = TRUE)

In [386]:
base_rf

Random Forest 

601 samples
 17 predictor
  2 classes: 'Class1', 'Class2' 

No pre-processing
Resampling: Cross-Validated (5 fold, repeated 2 times) 
Summary of sample sizes: 481, 480, 482, 480, 481, 480, ... 
Resampling results across tuning parameters:

  mtry  AUPRC      AUROC      Accuracy 
   2    0.8814762  0.9094983  0.8170681
   9    0.8685270  0.9023988  0.8269989
  17    0.8489735  0.8889804  0.8186792

Accuracy was used to select the optimal model using  the largest value.
The final value used for the model was mtry = 9.

In [387]:
base_rf$bestTune

,mtry
2,9


In [388]:
base_gbm <- train(x=train_set[,X],
      y=train_set[,Y],
      method = "gbm",
      trControl = ctrl,
      metric = "Accuracy",
      verbose = FALSE)

In [389]:
base_gbm

Stochastic Gradient Boosting 

601 samples
 17 predictor
  2 classes: 'Class1', 'Class2' 

No pre-processing
Resampling: Cross-Validated (5 fold, repeated 2 times) 
Summary of sample sizes: 481, 482, 480, 481, 480, 480, ... 
Resampling results across tuning parameters:

  interaction.depth  n.trees  AUPRC      AUROC      Accuracy 
  1                   50      0.8555833  0.8884066  0.8127909
  1                  100      0.8988430  0.9203945  0.8394240
  1                  150      0.9122358  0.9327877  0.8543761
  2                   50      0.8941225  0.9197675  0.8369240
  2                  100      0.9187043  0.9402239  0.8559870
  2                  150      0.9235559  0.9453067  0.8668763
  3                   50      0.9070120  0.9309715  0.8477093
  3                  100      0.9242064  0.9452826  0.8593623
  3                  150      0.9253258  0.9466818  0.8668835

Tuning parameter 'shrinkage' was held constant at a value of 0.1

Tuning parameter 'n.minobsinnode' was held

In [390]:
base_gbm$bestTune

,n.trees,interaction.depth,shrinkage,n.minobsinnode
9,150,3,0.1,10


# Stack

In [391]:
#Create folds for the stack
n=3

folds <- createFolds(train_set$Class,n)

train_set$foldID <- 0


for(i in 1:n){
    
   train_set[folds[[i]],]$foldID <- i
    
}

In [392]:
#Attach the rf output to the train set
train_set$rf <- 'Place_Holder'

for(i in 1:n){
    
    train <- train_set[train_set$foldID !=i,]
    
    
    tune_g <- base_rf$bestTune
    
    rf_train <- train(x=train[,X],
                 y=train[,Y],
                 method = "rf",
                 verbose = FALSE,
                 tuneGrid = tune_g,
                 allowParallel = TRUE)
    
    train_set[train_set$foldID == i,]$rf <- as.character( predict(rf_train, train_set[train_set$foldID == i,]) )
    
    #fit model to test data at each cv fold
    test_set[,paste0('rf',i)] <-  as.character( predict(rf_train, test_set) )


}

In [393]:
#Attach the gbm output to the train set
train_set$gbm <- 'Place_Holder'

for(i in 1:n){
    
    train <- train_set[train_set$foldID !=i,]
    
    
    tune_g <- base_gbm$bestTune
    
    gbm_train <- train(x=train[,X],
                 y=train[,Y],
                 method = "gbm",
                 verbose = FALSE,
                 tuneGrid = tune_g)
    
    train_set[train_set$foldID == i,]$gbm <- as.character( predict(gbm_train, train_set[train_set$foldID == i,]) )
    
    #fit model to test data at each cv fold
    test_set[,paste0('gbm',i)] <- as.character( predict(gbm_train, test_set) )

    
}

# Compare Models

In [394]:
#Campre the base rf, gbm to a glm and the stacked glm

In [395]:
#Fit the models to the training data

In [396]:
tune_g <- base_rf$bestTune

rf_fit <- train(x=train_set[,X],
      y=train_set[,Y],
      method = "rf",
      metric = "Accuracy",
      tuneGrid = tune_g,
      allowParallel = TRUE)

In [397]:
tune_g <- base_gbm$bestTune

gbm_fit <- train(x=train_set[,X],
      y=train_set[,Y],
      method = "gbm",
      metric = "Accuracy",
      tuneGrid = tune_g,
      verbose = FALSE)

In [398]:
glm_fit <- train(x=train_set[,X],
      y=train_set[,Y],
      method = "glm",
      metric = "Accuracy")

In [399]:
which(colnames(train_set) == 'rf')
which(colnames(train_set) == 'gbm')

[1] 20

[1] 21

In [400]:
glm_stack_fit <- train(x=train_set[,c(X,20,21)],
      y=train_set[,Y],
      method = "glm",
      metric = "Accuracy")

In [401]:
#Attach rf and gbm to the test_set for the model stacking

#base rf
test_set$rf <- ModeRow(test_set[,c('rf1','rf2','rf3')])

#base_gbm
test_set$gbm <- ModeRow(test_set[,c('gbm1','gbm2','gbm3')])

In [402]:
#Run the predictions on the test set
test_set$rf_base <- predict(rf_fit, test_set)

test_set$gbm_base <- predict(gbm_fit, test_set)

test_set$glm <- predict(glm_fit, test_set)

test_set$glm_stack <- predict(glm_stack_fit, test_set)

In [403]:
#Compare the accuracies of each
cat('rf base')
mean(test_set$Class == test_set$rf_base)

cat('rf cv')
mean(test_set$Class == test_set$rf)

cat('gbm base')

mean(test_set$Class == test_set$gbm_base)

cat('gbm cv')

mean(test_set$Class == test_set$gbm)

cat('glm')

mean(test_set$Class == test_set$glm)

cat('stacked glm')

mean(test_set$Class == test_set$glm_stack)

rf base

[1] 0.8421053

rf cv

[1] 0.8471178

gbm base

...

[1] 0.8947368

glm

[1] 0.8220551

stacked glm

[1] 0.887218